In [14]:
import os

In [15]:
pwd

'd:\\Personal\\Projects_Computer_Science_And_Engineering\\RaunakMLProjects\\End-to-end-ML-Project'

d:\Personal\Projects_Computer_Science_And_Engineering\RaunakMLProjects\End-to-end-ML-Project


d:\Personal\Projects_Computer_Science_And_Engineering\RaunakMLProjects\venv\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [16]:
os.chdir("../")

In [19]:
pwd

'd:\\Personal\\Projects_Computer_Science_And_Engineering\\RaunakMLProjects\\End-to-end-ML-Project'

In [79]:
from dataclasses import dataclass
from pathlib import Path
from typing import List

@dataclass(frozen=True)
class ModelTrainerConfig:
    # Define the fields with type annotations
    root_dir: Path
    train_data_path: Path
    test_data_path: Path   #from confi
    model_name: str     #from config
    random_state: int   #from param
    n_estimators: int   #from param
    target_column: str  #from schema
    selected_features: List[str]


In [80]:
from src.mlProject.constants import *
from src.mlProject.utils.common import read_yaml, create_directories

In [81]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.RandomForestClassifier
        schema =  self.schema.TARGET_COLUMN
        selected_features = self.schema.SELECTED_FEATURES

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            random_state = params.random_state,
            n_estimators = params.n_estimators, #change acc
            target_column = schema.name,
            selected_features=selected_features
            
        )

        return model_trainer_config

In [82]:
import os
from src.mlProject import logger
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import joblib


In [83]:
# should not be used below code 
# class ModelTrainer:
#     def __init__(self, config: ModelTrainerConfig):
#         self.config = config


    
#     def train(self):
#         train_data = pd.read_csv(self.config.train_data_path)
#         test_data = pd.read_csv(self.config.test_data_path)


#         train_x = train_data.drop([self.config.target_column], axis=1)
#         test_x = test_data.drop([self.config.target_column], axis=1)
#         train_y = train_data[[self.config.target_column]]
#         test_y = test_data[[self.config.target_column]]


#         rf = RandomForestClassifier(random_state=self.config.random_state, n_estimators=self.config.n_estimators)
#         rf.fit(train_x, train_y)

#         joblib.dump(rf, os.path.join(self.config.root_dir, self.config.model_name))

In [84]:
import pandas as pd
import joblib
import os
from sklearn.ensemble import RandomForestClassifier

class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        # Load data
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        # Get selected features and target column from the config
        selected_features = self.config.selected_features
        target_column = self.config.target_column

        # Ensure that the selected features are present in the data
        for feature in selected_features:
            if feature not in train_data.columns:
                raise ValueError(f"Feature column '{feature}' not found in the dataset")

        # Prepare train and test data (select only the defined feature columns)
        train_x = train_data[selected_features]
        test_x = test_data[selected_features]
        train_y = train_data[target_column]
        test_y = test_data[target_column]

        # Train the model (Random Forest in this case)
        rf = RandomForestClassifier(random_state=self.config.random_state, n_estimators=self.config.n_estimators)
        rf.fit(train_x, train_y)

        # Save the trained model
        joblib.dump(rf, os.path.join(self.config.root_dir, self.config.model_name),compress=("zlib",3)) #here aadded code for compression

        print("Model training complete.")


In [85]:
#pipeline
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2024-12-31 14:06:20,993: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-12-31 14:06:20,994: INFO: common: yaml file: params.yaml loaded successfully]
[2024-12-31 14:06:20,998: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-12-31 14:06:20,999: INFO: common: created directory at: artifacts]
[2024-12-31 14:06:21,000: INFO: common: created directory at: artifacts/model_trainer]
Model training complete.
